##### This module runs the ECG algorithm, tailored to the TimeWak datasets


In [1]:
import math
import numpy as np
import pandas as pd

import ECG.ECG_parameters as param
from ECG.ECG_robust import Preprocessing, WatermarkEmbedding, SignalAnalysis
from utils import get_mae

In [2]:
"""From TimeWak. load the real and fake data"""
iterations = 5

original_data = pd.read_csv('datasets/ETTh.csv', 
                            parse_dates=[0], 
                            date_format= '%d-%m-%y %H:%M')
print(len(original_data.iloc[:, 0]))


17420


In [9]:
"""Robust embedding"""

binary_ssn       = Preprocessing.convert_ssn_to_binary(param.user_ssn)
binary_ssn_split = Preprocessing.split_and_pad_binary_ssn(binary_ssn, binary_ssn_chunk_len = 4)
ssn_with_hamming = Preprocessing.apply_hamming_to_all_ssn_chunks(binary_ssn_split)

subsequence_length = param.subsequence_len_factor * len(ssn_with_hamming) # m = 3*l (from paper)

# n_timesteps = len(original_data.iloc[:, 0])
n_timesteps = math.floor(subsequence_length * param.num_subsequences) # from paper
ecg_signal  = original_data.iloc[:, 1].values

watermark_sequence     = WatermarkEmbedding._turn_watermark_to_nonbinary_sequence(ssn_with_hamming)
ecg_subsequences       = WatermarkEmbedding._split_signal_to_subsequences(ecg_signal, subsequence_length, n_timesteps)
# NOTE: ecg_subsequences is shorter than the original signal, because it tries to
# make all subsequences of the same length (which it doesnt have to)
# NOTE 2: why arent changes in the ECG_robust reflected here???

# print(len(ecg_subsequences), len(ecg_signal))

# length = 0
# for item in ecg_subsequences:
#     length += len(item)
#     print(len(item))
# print(length)    

watermarked_ecg_signal = WatermarkEmbedding.get_watermarked_subsequences(ecg_subsequences, watermark_sequence)

mae  = get_mae(ecg_signal, watermarked_ecg_signal)
mape = np.mean(np.abs((ecg_signal - watermarked_ecg_signal)/ecg_signal)) * 100
# print(f"Robust: MAE {mae}%, MAPE {mape}%")

should_we_plot = 0
SignalAnalysis.plot_robust_results(should_we_plot, ecg_signal, watermarked_ecg_signal)


ValueError: index can't contain negative values